In [1]:
import pandas as pd
import regex as re
from sklearn.model_selection import train_test_split

import numpy as np
from tqdm import tqdm
from numpy import dot
from numpy.linalg import norm

from sentence_transformers import SentenceTransformer

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

model = SentenceTransformer('jhgan/ko-sbert-multitask')

c:\Users\hyj\Anaconda3\envs\virenv39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = pd.read_csv('./preprocessed_data/mem_train.csv',index_col=[0])
valid_data = pd.read_csv('./preprocessed_data/mem_valid.csv',index_col=[0])
test_data = pd.read_csv('./preprocessed_data/mem_test.csv',index_col=[0])

train_data = train_data[['category','name','sent_cn']]
valid_data = valid_data[['category','name','sent_cn']]
test_data = test_data[['category','name','sent_cn']]

train_data['sent_cn'] = train_data['sent_cn'].apply(lambda x: eval(x))
valid_data['sent_cn'] = valid_data['sent_cn'].apply(lambda x: eval(x))
test_data['sent_cn'] = test_data['sent_cn'].apply(lambda x: eval(x))

train_data['sent_cn'] = train_data['sent_cn'].apply(lambda x: ' '.join(x))
valid_data['sent_cn'] = valid_data['sent_cn'].apply(lambda x: ' '.join(x))
test_data['sent_cn'] = test_data['sent_cn'].apply(lambda x: ' '.join(x))

In [3]:
df = pd.read_csv('./preprocessed_data/data.csv',index_col=[0])

print(len(df))
print(df['name'].nunique())

9000
9000


In [4]:
df['cn'] = df['cn'].apply(lambda x: x.replace('\t',''))
df['cn'] = df['cn'].apply(lambda x: x.replace('\xa0',''))

In [5]:
# 조 살려서 자르기
df['pre_cn'] = df['cn']
# 조 
pattern = '\\n([제|0-9 ]{1,}조[\(|가-힣\) |A-Z|a-z|0-9|,m]{1,})'
for i in range(len(df)):
    text = df['pre_cn'][i]
    matchedList = re.findall(pattern,text)
    for match in matchedList:
        text = text.replace(match,'\n?'+match+'\n')
    df['pre_cn'][i] = text

C:\Users\hyj\AppData\Local\Temp\ipykernel_16300\1750625931.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pre_cn'][i] = text


In [6]:
df['pre_cn'] = df['pre_cn'].apply(lambda x: x.replace('\n',''))
df['sent_jo'] = df['pre_cn'].apply(lambda x:x.split('?'))
df['sent_jo'] = df['sent_jo'].apply(lambda x: [i.strip() for i in x])
df['sent_jo'] = df['sent_jo'].apply(lambda x: [i for i in x if i not in {''}])

In [7]:
train_data = pd.merge(train_data,df[['category','name','sent_jo']],on=['name','category'],how='left')
train_data.columns = ['category','name','selected','sent_jo']

valid_data = pd.merge(valid_data,df[['category','name','sent_jo']],on=['name','category'],how='left')
valid_data.columns = ['category','name','selected','sent_jo']

test_data = pd.merge(test_data,df[['category','name','sent_jo']],on=['name','category'],how='left')
test_data.columns = ['category','name','selected','sent_jo']

In [8]:
test_data

,category,name,selected,sent_jo
0,19.증권사1,121_증권사1,제20조(수수료) 위탁판매 수수료와 관련하여 다음 방법중 하나를 선택할 수 있다.(...,"[증권대차거래약관, 제1조(약관의 적용) ① 이 약관은 유가증권시장 및 코스닥시장 ..."
1,"41. 통신, 방송서비스","056_통신,방송서비스","제12조(수신의 일시 중지)수신자가 출장, 입원 등 본인의 사유에 의해 수신을 일시...","[00000 이용약관 000 제1장 총칙, 제1조 목적 이 약관은 주식회사 000..."
2,1. 가맹계약_PDF,329_가맹계약,제 13 조 (부자재 및 제품 등 물품의 공급)을이 본 조 제1항에 정한 사항을 위...,[제 1 조 (계약의 목적)본 계약은 을이 갑의 ㅇㅇㅇ 상품을 상호 협의한 할인매장...
3,5. 임대차계약,384_임대차계약,제 5 조(계약의 해제 또는 해지)을이 다음 각호의 1에 해당하는 경우에는 갑은 이...,[제 1 조 (임대기간)1. 임대기간은 2020 년 09 월 28 일부터 2022 ...
4,9.은행여신약관,019_은행여신,제7조(기한전의 채무변제의무) ① 채무자에게 다음 각 호의 어느 하나에 해당하는 사...,"[00000 기본 약관(약관 신고 수리 일자 : 2019년 4월 22일), 제1조 ..."
...,...,...,...,...
656,18. 화재보험,079_화재보험,제11조 (계약 후 알릴 의무) ① 계약을 맺은 후 보험의 목적에 아래와 같은 사실...,"[00000 보통약관 제 1 관 목적 및 용어의 정의, 제1조(목적) 이 보험계..."
657,2. 공급계약,386_공급계약,제2조(계약의 해제) ① 갑 은 을 이 다음 각 호에 해당하는 행위를 하였을 때에...,[제1조(공급대금 및 납부방법) 갑은 표시재산을 아래방법으로 공급하고 을은 해당금액...
658,32.산후조리원,062_산후조리원,"제 6조 이용계약의 변경 및 조정 제 2항 입실시간은 입실일 10시 이후, 퇴실시간...",[000 이용약관본 약관은 000 제반시설물 이용에 관한 규정을 아래와 같이 정하고...
659,9.은행전자금융서비스,180_은행전자금융서비스,"제 18조 (변 경, 사 고 사 항 의 신 고 )ㅇ 보안카드의 분실, 도난 또는 각...",[제 1조 (목 적 )이 약관은 고객이 000(이하 회사 라 한다)의 전자금융서비...


In [9]:
def make_label(df,col1='selected',col2='sent_jo'):
    sent_sum_embed =  df[col1].apply(lambda x: model.encode(x))
    sent_cn_embed =  df[col2].apply(lambda x: model.encode(x))

    df['label'] = df[col2]
    for i, (cn_embeds, summ_embed) in enumerate(zip(sent_cn_embed,sent_sum_embed)):
        label = [0]*(cn_embeds.shape[0])
        cos_similarity = list(map(lambda x: cos_sim(x,summ_embed), cn_embeds))
        index = np.argmax(cos_similarity)
        if cos_similarity[index] >= 0.9:
            label[index]=1
        df['label'][i] = label
    return df

In [10]:
# 16분 
train_data = make_label(train_data)
valid_data = make_label(valid_data)
test_data = make_label(test_data)

In [11]:
valid_data

,category,name,selected,sent_jo,label
0,26. 국내외여행,185_국내외여행,제9조(최저행사인원 미 충족시 계약해제) ① 여행사는 최저행사인원이 충족되지 아니하...,"[해외여행표준약관, 제1조(목적)이 약관은 000의 여행사와 여행자가 체결한 국외여...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,12. 상해보험,093_상해보험,"제16조(사기에 의한 계약) 계약자 또는 피보험자가 대리진단, 약물사용을 수단으로 ...","[00000 보통약관 제1관 목적 및 용어의 정의, 제1조(목적) 이 보험계약(이...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,22. 여객운송,123_여객운송,제72조 (여객에 의한 계약변경) 1 여객의 사정에 의하여 계약서에 기재된 사항대로...,[여객운송약관 ( 규정 제47호 1996. 12. 6 ) 개정 1999. 11. ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,40. 택배,060_택배,제21조(책임의 특별소멸 사유와 시효) ① 운송물의 일부 멸실 또는 훼손에 대한 회...,"[공정거래위원회 퀵서비스 표준약관 제 1 장 총 칙, 제1조 (목적) 이 약관은 택...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,10. 전자결제수단,098_전자결제수단,제15조 (등록상표의 사용) ① 갑은 을이 지정한 등록상표 및 로고를 사용하여야 한...,"[000 00000 약관 적용 제1장 총 칙, 제1조(목적) 이 약관은 000사(...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
655,32. 산후조리원,292_산후조리원,제 2조 (입실 후 계약의 해제) ① 이용자는 입실 후에 사업자의 책임 있는 사유로...,[제 1조 (약관의 목적) 본 약관은 000(이하 본원이라 명함)이 제공하는 모든 ...,"[0, 0, 0, 0, 0, 0, 0, 1]"
656,35. 생명보험,112_생명보험,제26조(청약의 철회) ① 계약자는 공제증권을 받은 날부터 15일 이내에 그 청약을...,"[무배당 어부바신협임직원보장공제(간편가입) 보통약관 직원형 1종(간편가입형), 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
657,26. 국내외여행,199_국내외여행,제12조 (여행요금의 변경) ① 국외여행을 실시함에 있어서 이용운송․숙박기관에 지급...,[제1조 (목적) 이 약관은 주식회사 베테랑이경수여행사와 여행자가 체결한 국외여행계...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
658,2. 공급계약,309_공급계약,"제10조 ( 품명 )을이 갑에게 공급하는 시멘트는 을이 생산하는 각종시멘트, 동종의...",[제 1 조 (계약의 목적)본 계약은 을이 갑의 000 상품을 상호 협의한 할인매장...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


In [12]:
train_labeled_df = train_data.loc[train_data['label'].apply(lambda x: sum(x))!=0]
train_droped_df = train_data.loc[train_data['label'].apply(lambda x: sum(x))==0]

train_labeled_df.reset_index(drop=True,inplace=True)
train_droped_df.reset_index(drop=True,inplace=True)

valid_labeled_df = valid_data.loc[valid_data['label'].apply(lambda x: sum(x))!=0]
valid_droped_df = valid_data.loc[valid_data['label'].apply(lambda x: sum(x))==0]

valid_labeled_df.reset_index(drop=True,inplace=True)
valid_droped_df.reset_index(drop=True,inplace=True)

test_labeled_df = test_data.loc[test_data['label'].apply(lambda x: sum(x))!=0]
test_droped_df = test_data.loc[test_data['label'].apply(lambda x: sum(x))==0]

test_labeled_df.reset_index(drop=True,inplace=True)
test_droped_df.reset_index(drop=True,inplace=True)

In [13]:
print(len(train_data))
print(len(train_labeled_df))

print(len(valid_data))
print(len(valid_labeled_df))

print(len(test_data))
print(len(test_labeled_df))

5281
5281
660
660
661
661


In [14]:
def make_article_df(df):
    row = 0
    article_df = pd.DataFrame(columns=['name', 'sent_jo', 'label'])

    for row in tqdm(range(len(df))):
        tmp_df1 = pd.DataFrame(df['sent_jo'][row])
        tmp_df2 = pd.DataFrame(df['label'][row])
        row_df = pd.concat([tmp_df1,tmp_df2],axis=1)
        row_df.columns = ['sent_jo','label']
        row_df['name'] = df['name'][row]
        article_df = pd.concat([article_df,row_df])
        
    return article_df

In [15]:
train_article_df = make_article_df(train_labeled_df)
valid_article_df = make_article_df(valid_labeled_df)
test_article_df = make_article_df(test_labeled_df)

100%|██████████| 661/661 [00:01<00:00, 455.82it/s]


In [16]:
valid_article_df

,name,sent_jo,label
0,185_국내외여행,해외여행표준약관,0
1,185_국내외여행,제1조(목적)이 약관은 000의 여행사와 여행자가 체결한 국외여행계약의 세부 이행 ...,0
2,185_국내외여행,제2조(여행업자와 여행자 의무)① 여행업자는 여행자에게 안전하고 만족스러운 여행서비...,0
3,185_국내외여행,"제3조(용어의 정의)여행의 종류 및 정의, 해외여행수속대행업의 정의는 다음과 같다....",0
4,185_국내외여행,"제4조(계약의 구성)① 여행계약서(붙임)와 여행약관, 여행일정표(또는 여행설명서)②...",0
...,...,...,...
17,158_화물운송,제16조(사고증명서 발행) 사업자는 제14조의 사고가 발생하여 화주가 사고 증...,0
18,158_화물운송,제17조(배상 면책) 사업자는 다음 경우에는 배상책임을 지지 아니한다. ...,0
19,158_화물운송,제18조(사고통지 의무) 사업자는 수탁화물에 사고가 발생하였을 경우에는 지체 ...,0
20,158_화물운송,제19조(화물의 운임) ① 운임계산은 건설교통부장관이 정한 요율과 기준에 의하여...,0


In [17]:
train_article_df['split'] = 'train'
valid_article_df['split'] = 'valid'
test_article_df['split'] = 'test'

concated_article_df = pd.concat([test_article_df,valid_article_df,train_article_df])
print(len(concated_article_df))

# 문장이랑 라벨이 같은 건 제거
concated_article_df = concated_article_df.drop_duplicates(['sent_jo','label'],ignore_index=True)
print(len(concated_article_df))

175180
91806


In [18]:
print(concated_article_df['label'].value_counts())

concated_article_df = concated_article_df.sort_values(['label'],ascending=False) # 1이 먼저 오도록 정렬
concated_article_df = concated_article_df.drop_duplicates('sent_jo',ignore_index=True) # 문장이 같은 것 제거
print(concated_article_df['label'].value_counts()) # 위와 비교했을 때 1 개수는 변화가 있으면 안됨.

0    85249
1     6557
Name: label, dtype: int64
0    85008
1     6557
Name: label, dtype: int64


In [19]:
print(len(concated_article_df))
print(len(concated_article_df.drop_duplicates(['sent_jo']))) # 중복되는 문장 없음

91565
91565


In [20]:
print(train_data['name'].nunique())
print(concated_article_df.loc[concated_article_df['split']=='train']['name'].nunique())

5226
5211


In [21]:
print(valid_data['name'].nunique())
print(concated_article_df.loc[concated_article_df['split']=='valid']['name'].nunique())

657
655


In [22]:
print(test_data['name'].nunique())
print(concated_article_df.loc[concated_article_df['split']=='test']['name'].nunique())

660


660


In [23]:
train_article_df = concated_article_df.loc[concated_article_df['split']=='train']
valid_article_df = concated_article_df.loc[concated_article_df['split']=='valid']
test_article_df = concated_article_df.loc[concated_article_df['split']=='test']

In [24]:
train_article_df.drop(['split'],axis=1,inplace=True)
valid_article_df.drop(['split'],axis=1,inplace=True)
test_article_df.drop(['split'],axis=1,inplace=True)

C:\Users\hyj\AppData\Local\Temp\ipykernel_16300\2994867359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_article_df.drop(['split'],axis=1,inplace=True)
C:\Users\hyj\AppData\Local\Temp\ipykernel_16300\2994867359.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_article_df.drop(['split'],axis=1,inplace=True)
C:\Users\hyj\AppData\Local\Temp\ipykernel_16300\2994867359.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [25]:
train_article_df.to_csv('./preprocessed_data//article_train.csv')
valid_article_df.to_csv('./preprocessed_data/article_valid.csv')
test_article_df.to_csv('./preprocessed_data//article_test.csv')